In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [79]:
import numpy as np # Importing numpy for performing array and matrix operations 
import pandas as pd # Importing pandas to read data in dataframe

# Libraries to plot the distribution of classes 
import seaborn as sns
import matplotlib.pyplot as plt
import datetime #I will use this to Convert ISO8601 format to unix time
from matplotlib.pyplot import figure

import os
import random

import time #I will use this to Convert ISO8601 format to unix time
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

from sklearn import linear_model  #Importing linear models like logistics regression and SVC
from sklearn.preprocessing import LabelEncoder  # To convert every class  into numeric labels 
from sklearn.metrics import f1_score       # to calculate F1 score 
from sklearn.metrics import classification_report # to know every metric
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
from tensorflow.keras import backend as K
from collections import deque
from tensorflow.keras.layers import Layer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Input,concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, Model, backend as K

In [ ]:
terminal='S56-T3'
path='/content/drive/MyDrive/AAIC_Assignment/EV_Trained Models/'+terminal
reconstructed_model = keras.models.load_model(path)

In [ ]:
dct_labels = {}

In [ ]:
dct_labels['S7-t1']='Available'

In [ ]:
print(dct_labels)

{'S7-t1': 'Available'}


In [80]:
def final_fun1(data):
  '''
  This function takes the Raw data as input and after proprocessing and featuer
  engineering returns the predictions made by differnt models

  '''
  dct = {}
  def date_time(data):
    
    '''This function takes the datetime data and convert that 
       into its individual components like month, day, week hr and minute

    '''
   # Initializing Empty lists
    month=[]
    week=[]
    date=[]
    hr=[]
    minute=[]

# I have made timestamp column as an index of the dataframe for easy plotting operations
    for i in data.index: # For every index of the dataframe
    #print(i)
    #break
    #dt = datetime.datetime.strptime(i, "%Y-%m-%dT%H:%M:%S%z")
        dt=i
    # Appending the month, day, weekday, hour and minute in lists
        
        month.append(dt.month)
        week.append(dt.weekday()) #Monday is 0 and Sunday is 6
        date.append(dt.day)
        hr.append(dt.hour)
        minute.append(dt.minute)
# making X as a data frame 
    return pd.DataFrame(list(zip( month, week, date, hr, minute)), columns =['Month_name', 'Week_number', 'Date', 'Hour', 'Minute'])

  X=date_time(data)
  terminal_lst=['S7-T1', 'S2-T1', 'S19-T1', 'S56-T3', 'S85-T3','S16-T3', 'S16-T1',
                     'S94-T3', 'S28-T1', 'S62-T3', 'S14-T3', 'S8-T1', 'S8-T3', 'S34-T3',
                     'S6-T2', 'S29-T3', 'S64-T1', 'S11-T1', 'S76-T3', 'S59-T1']
  for terminal in terminal_lst:
    if terminal in ['S7-T1', 'S19-T1', 'S85-T3', 'S28-T1', 'S62-T3']:
      text='Down'
      dct[terminal]=text
    else:
      path='/content/drive/MyDrive/AAIC_Assignment/EV_Trained Models/'+terminal
      reconstructed_model = keras.models.load_model(path)
      y_pred=reconstructed_model.predict(X)
      val_predict=np.argmax(y_pred, axis=1)
      if val_predict==0:
            text='Available'
            dct[terminal]=text
      elif val_predict==1:
            text='Charging'
            dct[terminal]=text
      elif val_predict==2:
            text='Passive'
            dct[terminal]=text
      elif val_predict==3:
            text='Offline'
            dct[terminal]=text
      else:
            text='Down' 
            dct[terminal]=text

  return dct


In [95]:
data= pd.read_csv('/content/drive/MyDrive/AAIC_Assignment/yrandom.csv', parse_dates=['timestamp'],
index_col= ['timestamp'],nrows=10)

In [82]:
data.head()

,S7-T1,S2-T1,S19-T1,S56-T3,S85-T3,S16-T3,S16-T1,S94-T3,S28-T1,S62-T3,S14-T3,S8-T1,S8-T3,S34-T3,S6-T2,S29-T3,S64-T1,S11-T1,S76-T3,S59-T1,S58-T1,S58-T3,S21-T3,S96-T3,S30-T1,S77-T1,S35-T3,S71-T3,S89-T1,S51-T3,S51-T1,S38-T3,S79-T1,S79-T3,S33-T3,S33-T1,S50-T3,S1-T3,S49-T1,S60-T3,...,S71-T2,S36-T2,S79-T2,S16-T2,S94-T2,S9-T2,S21-T1,S92-T2,S11-T2,S51-T2,S60-T2,S19-T2,S62-T2,S31-T2,S96-T2,S27-T2,S23-T2,S28-T2,S37-T2,S39-T2,S68-T2,S56-T2,S41-T2,S35-T2,S7-T2,S75-T2,S20-T2,S95-T2,S5-T2,S2-T2,S47-T2,S65-T2,S32-T2,S21-T2,S13-T2,S97-T2,S25-T1,S25-T2,S98-T2,S99-T2
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-11-09 00:00:00+00:00,Offline,Available,Charging,Charging,Available,Available,Passive,Available,Charging,Charging,Offline,Passive,Available,Down,Available,Charging,Charging,Passive,Offline,Available,Down,Down,Offline,Offline,Offline,Available,Offline,Down,Available,Down,Offline,Available,Available,Charging,Passive,Charging,Charging,Passive,Offline,Charging,...,Available,Down,Offline,Available,Available,Down,Charging,Down,Down,Down,Available,Down,Offline,Passive,Charging,Offline,Charging,Charging,Passive,Passive,Charging,Down,Passive,Offline,Down,Charging,Down,Passive,Passive,Passive,Down,Down,Available,Passive,Offline,Charging,Passive,Offline,Charging,Offline


In [83]:

rs=final_fun1(data)

In [84]:
rs

{'S11-T1': 'Available',
 'S14-T3': 'Offline',
 'S16-T1': 'Offline',
 'S16-T3': 'Offline',
 'S19-T1': 'Down',
 'S2-T1': 'Offline',
 'S28-T1': 'Down',
 'S29-T3': 'Available',
 'S34-T3': 'Offline',
 'S56-T3': 'Available',
 'S59-T1': 'Offline',
 'S6-T2': 'Available',
 'S62-T3': 'Down',
 'S64-T1': 'Available',
 'S7-T1': 'Down',
 'S76-T3': 'Available',
 'S8-T1': 'Offline',
 'S8-T3': 'Offline',
 'S85-T3': 'Down',
 'S94-T3': 'Available'}

In [96]:
def final_fun2(data):
  '''
  This function takes the data as daatframe with time series and 
  terminal names with thier corresponding labels 
  Only 20 terminals are allowed
  The Timeseries must be parsed and should be an index column 
  All other columns are terminal columns

  This fuction will return the Weighted F1 score on the test data

  '''



  def date_time(data):
    
    '''This function takes the datetime data and convert that 
       into its individual components like month, day, week hr and minute

    '''
    # Initializing Empty lists
    month=[]
    week=[]
    date=[]
    hr=[]
    minute=[]

    # I have made timestamp column as an index of the dataframe for easy plotting operations
    for i in data.index: # For every index of the dataframe
        #print(i)
        #break
        #dt = datetime.datetime.strptime(i, "%Y-%m-%dT%H:%M:%S%z")
        dt=i
        # Appending the month, day, weekday, hour and minute in lists
        
        month.append(dt.month)
        week.append(dt.weekday()) #Monday is 0 and Sunday is 6
        date.append(dt.day)
        hr.append(dt.hour)
        minute.append(dt.minute)
    # making X as a data frame 
    return pd.DataFrame(list(zip( month, week, date, hr, minute)), columns =['Month_name', 'Week_number', 'Date', 'Hour', 'Minute'])
  X=date_time(data)
    
  def label_encoder(column):
      '''
      This function help us to Encode the class labels of 
      the corresponding terminal into their respective 
      numeric labels
      '''
      if column =='Available': # Available ==0
          return 0
      elif column=='Charging': # charging==1
          return 1
      elif column=='Passive': # Passive==2
          return 2
      elif column=='Offline': # offline==3
          return 3
      else: return 4    # Down==4
# utilize it along with apply method

  warnings.filterwarnings("ignore")
  f1_state_terminal=[0,0,0,0,0, 0] # order available, passive, charging, offline, down
  f1_state_terminal_test=[0,0,0,0,0, 0] # order available, passive, charging, offline, down
  label_count=[0,0,0,0,0,0]
  for i in range(0,20):
 
    terminal= data.columns[i] # Extracting the column names from the dataframe
      #num_class=data[terminal].unique() # Unique labels present in the dataframe column
      #print(terminal)
    weekend=data[terminal].apply(label_encoder)
    num_class=weekend.unique()
    num_class.sort()

    
    if terminal not in ['S7-T1', 'S19-T1', 'S85-T3', 'S28-T1', 'S62-T3']:
      
      #print("I am here", terminal)

      y=np.array(weekend).reshape(-1,1) # label encoding class labels
      y_train1 = to_categorical(y, 5)

      #print(y_train1.shape)
      #print(y_test1.shape)   
            # define model   
      path='/content/drive/MyDrive/AAIC_Assignment/EV_Trained Models/'+terminal
      reconstructed_model = keras.models.load_model(path)
      y_pred=reconstructed_model.predict(X)
      
      val_predict=np.argmax(y_pred, axis=1)
      val_target=np.argmax(y_train1, axis=1) 
      state_f1=f1_score(val_target, val_predict, average=None)
           
      idx=0
      for label in num_class:
          #print("more than 1",num_class)
          f1_state_terminal[label]+=state_f1[idx]
          label_count[label]+=1
          idx+=1
          #print('passed from if ')
              
    else:    
      #print(terminal)
      for label in num_class:
          #print('1 class',num_class)
          f1_state_terminal[label]+=1 # Adding F1 score as 1 to the state where a terminal is in only one mode
          label_count[label]+=1
          #print('passed from else')
  av_count=0
  ch_count=0
  pa_count=0
  do_count=0                             
  of_count=0
  for i in tqdm(range(0,20)):
    terminal= data.columns[i] # Extracting the column names from the dataframe
    labels=data[terminal]
    for i in labels:
        #print(i)
        if i=='Available':
            av_count+=1
        if i=='Charging':
            ch_count+=1
        if i=='Passive':
            pa_count+=1
        if i=='Down':
            do_count+=1
        if i=='Offline':
            of_count+=1
  tot_count=av_count+ch_count+pa_count+do_count+of_count
  #print(tot_count)
  prob_list=[0,0,0,0,0]# order available, passive, charging, offline, down
  prob_list[0]=av_count/tot_count
  prob_list[1]=pa_count/tot_count
  prob_list[2]=ch_count/tot_count
  prob_list[3]=of_count/tot_count
  prob_list[4]=do_count/tot_count    

  #print('The counts of the class labels in train data', label_count)
#print("The Combined F1 scores of Terminals ",f1_state_terminal) # order available, passive, charging, offline, down
  for i in range(len(f1_state_terminal)-1):
    f1_state_terminal[i]/=20


  f1_final=0
  #print(" The probabilities of the train states",prob_list)
  for i in range(len(prob_list)):
    f1_final+=prob_list[i]*f1_state_terminal[i]

  return f1_final

In [97]:
a=final_fun2(data)

100%|██████████| 20/20 [00:00<00:00, 3106.89it/s]


In [98]:
a

0.3087021978021978